# Upload a image of a bear and have my model classify it for you!
#### Note: Can only classify Teddy, Black, and Grizzly bears.

In [2]:
from fastbook import *
from fastai.vision.widgets import *

path = Path()
learn_inf = load_learner(path/'export.pkl')

In [6]:
btn_upload = widgets.FileUpload()
out_pl = widgets.Output()
lbl_pred = widgets.Label()
btn_run = widgets.Button(description='Classify')

In [4]:
def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = learn_inf.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

In [7]:
btn_run.on_click(on_click_classify)

In [10]:
VBox([widgets.Label('Select your bear!'), 
      btn_upload, btn_run, out_pl, lbl_pred])